# 垃圾分拣玩法  Garbage sorting play

### 导入头文件 Import head file

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import Arm_Lib
import cv2 as cv
import threading
from time import sleep
from dofbot_config import *
import ipywidgets as widgets
from IPython.display import display
from garbage_identify import garbage_identify

### 初始化机械臂位置 Initialize DOFBOT position

In [ ]:
target = garbage_identify()
calibration = Arm_Calibration()
arm = Arm_Lib.Arm_Device()
num=0
dp = []
msg = {}
xy = [90, 135]
threshold = 140
model = "General"
# XYT参数路径  XYT Parameter path
XYT_path="/home/jetson/dofbot_ws/src/dofbot_garbage_yolov5/XYT_config.txt"
try: xy, thresh = read_XYT(XYT_path)
except Exception: print("Read XYT_config Error !!!")

### 创建实例,初始化参数 Create the instance and initialize the parameters

In [ ]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [xy[0], xy[1], 0, 0, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件 Creating widget

In [ ]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 调整滑杆 Adjust the slider
joint1_slider      = widgets.IntSlider(description='joint1 :'   ,    value=xy[0]     , min=70 , max=110, step=1, orientation='horizontal')
joint2_slider      = widgets.IntSlider(description='joint2 :'   ,    value=xy[1]     , min=115, max=155, step=1, orientation='horizontal')
threshold_slider   = widgets.IntSlider(description='threshold :',    value=threshold , min=0  , max=255, step=1, orientation='horizontal')

# 进入标定模式  Enter calibration mode
calibration_model  = widgets.Button(description='calibration_model',  button_style='primary', layout=button_layout)
calibration_ok     = widgets.Button(description='calibration_ok',     button_style='success', layout=button_layout)
calibration_cancel = widgets.Button(description='calibration_cancel', button_style='danger', layout=button_layout)

# 目标检测抓取
target_detection   = widgets.Button(description='target_detection', button_style='info', layout=button_layout)
grap = widgets.Button(description='grap', button_style='success', layout=button_layout)
# 退出  exit
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
garbage_identify = widgets.VBox(
    [joint1_slider, joint2_slider, threshold_slider, calibration_model, calibration_ok, calibration_cancel, target_detection, grap,exit_button],
    layout=widgets.Layout(align_self='center'));
controls_box = widgets.HBox([imgbox, garbage_identify], layout=widgets.Layout(align_self='center'))

### 标定回调 Calibration callback function

In [ ]:
def calibration_model_Callback(value):
    global model
    model = 'Calibration'
    with output: print(model)
def calibration_OK_Callback(value):
    global model
    model = 'calibration_OK'
    with output: print(model)
def calibration_cancel_Callback(value):
    global model
    model = 'calibration_Cancel'
    with output: print(model)
calibration_model.on_click(calibration_model_Callback)
calibration_ok.on_click(calibration_OK_Callback)
calibration_cancel.on_click(calibration_cancel_Callback)

### 模式切换   switching mode

In [ ]:
def target_detection_Callback(value):
    global model
    model = 'Detection'
    with output: print(model)
def grap_Callback(value):
    global model
    model = 'Grap'
    with output: print(model)
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
target_detection.on_click(target_detection_Callback)
grap.on_click(grap_Callback)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [ ]:
def camera():
    global color_hsv,model,dp,msg,color_list,HSV_name
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    # Be executed in loop when the camera is opened normally 
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            img = cv.resize(img, (640, 480))
            xy=[joint1_slider.value,joint2_slider.value]
            if model == 'Calibration':
                _, img = calibration.calibration_map(img,xy,threshold_slider.value)
            if model == 'calibration_OK':
                try: write_XYT(XYT_path,xy, threshold_slider.value)
                except Exception: print("File XYT_config Error !!!")
                dp, img = calibration.calibration_map(img,xy,threshold_slider.value)
                model="General"
            if len(dp) != 0: img = calibration.Perspective_transform(dp, img)
            if model == 'calibration_Cancel':  
                dp = []
                msg= {}
                model="General"
            if len(dp)!= 0 and  model == 'Detection': img, msg = target.garbage_run(img)
            if len(msg)!= 0 and model == 'Grap':
                threading.Thread(target=target.garbage_grap, args=(msg, xy,)).start()
                msg={}
                model="Detection"
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动  Start

In [ ]:
display(controls_box,output)
threading.Thread(target=camera, ).start()